In [ ]:
!pip install sklearn_crfsuite

     |████████████████████████████████| 743 kB 3.2 MB/s 


In [ ]:
!pip install seqeval

     |████████████████████████████████| 43 kB 814 kB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=b8d95a2c24b4685fa3661dbd9a09ffc6d92407d6c8dc43938107330f42dc43a0
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [ ]:
!pip install eli5

     |████████████████████████████████| 106 kB 4.2 MB/s 


In [ ]:
import pandas as pd
import numpy as np

import os
import re
import string
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter 

from sklearn_crfsuite import CRF
import eli5
#from sklearn.metrics import make_scorer, classification_report

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import RepeatedKFold, cross_val_score

from sklearn_crfsuite.metrics import flat_classification_report, flat_f1_score, flat_precision_score, flat_recall_score
import scipy.stats
from sklearn.metrics import make_scorer
#from seqeval.metrics import precision_score, recall_score, f1_score, classification_report


In [ ]:
with open('ADE_POS_to_NER.pickle', 'rb') as handle:
    df = pickle.load(handle)

df = df[['sentence', 'Word', 'tag']]
df[:5]

,sentence,Word,tag
0,1. Changes in the plasma cortisol level were r...,1.,O
1,1. Changes in the plasma cortisol level were r...,Changes,I_ADR
2,1. Changes in the plasma cortisol level were r...,in,I_ADR
3,1. Changes in the plasma cortisol level were r...,the,I_ADR
4,1. Changes in the plasma cortisol level were r...,plasma,I_ADR


### Get sentences with word's label

In [ ]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w,  t) for w,  t in zip(s['Word'].values.tolist(), 
                                                           
                                                           s['tag'].values.tolist())]
        self.grouped = self.data.groupby('sentence').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

In [ ]:
#Displaying one full sentence
getter = SentenceGetter(df)

#sentence with its tag
sentences = getter.sentences

### Total sentences, words, index

In [ ]:
getter = SentenceGetter(df)
sentences = getter.sentences

words = list(set(df["Word"].values))
words.append("ENDPAD")
n_words = len(words)
print('Total unique words: ', n_words)

max_len = max([len(s) for s in sentences])
print ('Maximum sequence length:', max_len)

tags = list(set(df["tag"].values))
print('Tags are : ', tags)
num_tag = df['tag'].nunique()
print('Number of tags : ', num_tag)

Total unique words:  8715
Maximum sequence length: 94
Tags are :  ['O', 'I_Drug', 'I_ADR']
Number of tags :  3


In [ ]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        #'postag': postag,
        #'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            #'-1:postag': postag1,
            #'-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            #'+1:postag': postag1,
            #'+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
  return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
  return [label for token, label in sent]

def sent2tokens(sent):
  return [token for token, label in sent]

In [ ]:
#X = [sent2features(s) for s in sentences]
#y = [sent2labels(s) for s in sentences]
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=99)

In [ ]:
train_sentences, test_sentences = train_test_split(sentences, test_size=0.3, random_state=42)

#Train set
X_train = [sent2features(s) for s in train_sentences]
y_train = [sent2labels(s) for s in train_sentences]

#Test set
X_test = [sent2features(s) for s in test_sentences]
y_test = [sent2labels(s) for s in test_sentences]

In [ ]:
#train_sentences[0]

In [ ]:
'''
crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         max_iterations = 100,
         all_possible_transitions = False)
crf.fit(X_train, y_train)

eli5.show_weights(crf, top=3)
'''

"\ncrf = CRF(algorithm = 'lbfgs',\n         c1 = 0.1,\n         c2 = 0.1,\n         max_iterations = 100,\n         all_possible_transitions = False)\ncrf.fit(X_train, y_train)\n\neli5.show_weights(crf, top=3)\n"

### Fine tunning hyper-parameters +  Cross-validation 

In [ ]:
# define fixed parameters and parameters to search
crf = CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(flat_f1_score, average='weighted', labels=tags)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=5,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  1.4min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=CRF(algorithm='lbfgs', all_possible_states=None,
                                 all_possible_transitions=True, averaging=None,
                                 c=None, c1=None, c2=None,
                                 calibration_candidates=None,
                                 calibration_eta=None,
                                 calibration_max_trials=None,
                                 calibration_rate=None,
                                 calibration_samples=None, delta=None,
                                 epsilon=None, error_sensitive=None, gamma=None,
                                 keep_...
                   iid='deprecated', n_iter=5, n_jobs=-1,
                   param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fd1c18dde80>,
                                        'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fd1c18b1048>},
                 

In [ ]:
# crf = rs.best_estimator_
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.4808461315123546, 'c2': 0.001129524235590414}
best CV score: 0.9066450994456607
model size: 0.47M


In [ ]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(flat_classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

       I_ADR     0.8374    0.7426    0.7872      3850
      I_Drug     0.8775    0.8734    0.8754      1911
           O     0.9392    0.9609    0.9499     19319

    accuracy                         0.9207     25080
   macro avg     0.8847    0.8589    0.8708     25080
weighted avg     0.9189    0.9207    0.9193     25080



In [ ]:
print('Precision on Test set : {:.2%}'.format(flat_precision_score(y_test, y_pred, average='weighted', labels=tags)))
print('Recall on Test set : {:.2%}'.format(flat_recall_score(y_test, y_pred, average='weighted', labels=tags)))
print('F1-socre on Test set : {:.2%}'.format(flat_f1_score(y_test, y_pred, average='weighted', labels=tags)))

Precision on Test set : 91.89%
Recall on Test set : 92.07%
F1-socre on Test set : 91.93%


In [ ]:
eli5.show_weights(crf, top=3)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


In [ ]:
#SAVE Model
pickle.dump(crf, open('NLP_Drug_NER_CRF.pickle', 'wb'))

In [ ]:
# load the model from disk
loaded_model = pickle.load(open('NLP_Drug_NER_CRF.pickle', 'rb'))

result = flat_classification_report(y_test, loaded_model.predict(X_test), digits=4)
print(result)

              precision    recall  f1-score   support

       I_ADR     0.8374    0.7426    0.7872      3850
      I_Drug     0.8775    0.8734    0.8754      1911
           O     0.9392    0.9609    0.9499     19319

    accuracy                         0.9207     25080
   macro avg     0.8847    0.8589    0.8708     25080
weighted avg     0.9189    0.9207    0.9193     25080



### What our classifier learned ?

In [ ]:
def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))
print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(3))
print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-6:])

Top likely transitions:
O      -> O       2.058832
I_ADR  -> I_ADR   1.398106
I_Drug -> I_Drug  1.162295

Top unlikely transitions:
I_Drug -> O       -0.251219
I_ADR  -> O       -0.393158
O      -> I_Drug  -0.564575
O      -> I_ADR   -1.722634
I_ADR  -> I_Drug  -2.270160
I_Drug -> I_ADR   -2.363947


* It is likely that an Out entity ('O') will be followed by a token inside Out entity ('O'), I_ADR followed by I_ADR, I_Drug followed by I_Drug
* Transitions from I_ADR to I_Drug , from I_Drug to I_ADR, O to I_ADR are penalized hugely.

### Prediction on a random test phrase

In [ ]:
test_phrases = [" ".join([s[0] for s in sent]) for sent in test_sentences]
test_phrases[0]

'Gabapentin toxicity in renal failure: the importance of dose adjustment'

In [ ]:
# At every execution model picks some random test sample from test set.
i = np.random.randint(0,len(X_test)) # choose a random number between 0 and len(X_te)
X_to_test = [sent2features(test_sentences[i])]
y_true_test = [sent2labels(test_sentences[i])]

print("Sample number {} of {} (Test Set)".format(i, len(X_test)))
print(test_phrases[i])
# Visualization
print("{:20}||{:6}||{}".format("Word", "True", "Pred"))
print(30 * "=")
for w, t, pred in zip(test_phrases[i].split(), y_true_test[0], crf.predict(X_to_test)[0]):
    if w != 0:
        print("{:20}: {:6} {}".format(w, t, pred))

Sample number 1066 of 1282 (Test Set)
An asymptomatic HIV - infected woman experienced right - sided renal colicky pain during treatment with indinavir
Word                ||True  ||Pred
An                  : O      O
asymptomatic        : O      O
HIV                 : O      O
-                   : O      O
infected            : O      O
woman               : O      O
experienced         : O      O
right               : O      O
-                   : O      O
sided               : O      O
renal               : I_ADR  I_ADR
colicky             : I_ADR  I_ADR
pain                : I_ADR  I_ADR
during              : O      O
treatment           : O      O
with                : O      O
indinavir           : I_Drug I_Drug
